<a href="https://colab.research.google.com/github/146790g/pytorch_advanced/blob/master/Ch11_5_List11_10_11_11_11_12_Training_02_Correct_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install SimpleITK
!pip install diskcache 

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Pytorch CT Lung Cancer/deep-learning-with-pytorch-ja-main/p2ch11')

###Pathを通す

In [ ]:
import os
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Pytorch CT Lung Cancer/deep-learning-with-pytorch-ja-main')

# **①モジュールを呼び込む**

In [ ]:
import argparse
import datetime
import os
import sys

import numpy as np

from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader

from util.util import enumerateWithEstimate
from dsets2 import LunaDataset
from util.logconf import logging
from model import LunaModel


In [ ]:
#!wget -P ./data/part2/luna https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/part2/luna/annotations.csv
#!wget -P ./data/part2/luna https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/part2/luna/candidates.csv

# **①argparse.Namespaceの作成**

In [ ]:
import argparse
import os
from util import util
import torch

class BaseOptions():
    def __init__(self):
        self.parser = argparse.ArgumentParser()

    def initialize(self):
        self.parser.add_argument('--num-workers',
            help='Number of worker processes for background data loading',
            default=8,
            type=int,
        )
        self.parser.add_argument('--batch-size',
            help='Batch size to use for training',
            default=4,
            type=int,
        )
        self.parser.add_argument('--epochs',
            help='Number of epochs to train for',
            default=1,
            type=int,
        )

        self.parser.add_argument('--tb-prefix',
            default='p2ch11',
            help="Data prefix to use for Tensorboard run. Defaults to chapter.",
        )

        self.parser.add_argument('comment',
            help="Comment suffix for Tensorboard run.",
            nargs='?',
            default='dwlpt',
        )
    def parse(self):
        self.opt = self.parser.parse_args(args=[])
        args = vars(self.opt)

        print('------------ Options -------------')
        for k, v in sorted(args.items()):
            print('%s: %s' % (str(k), str(v)))
        print('-------------- End ----------------')
        return self.opt

config = BaseOptions()
config.initialize()
opt=config.parse()
print(config.opt.epochs)
print(config.opt.comment)
print(config.opt.num_workers)

------------ Options -------------
batch_size: 4
comment: dwlpt
epochs: 1
num_workers: 8
tb_prefix: p2ch11
-------------- End ----------------
1
dwlpt
8


In [ ]:
print(opt)

Namespace(batch_size=4, comment='dwlpt', epochs=1, num_workers=8, tb_prefix='p2ch11')


# **②DataSet Class作成**

In [ ]:
import os
dataset = LunaDataset(
    val_stride=10,
    isValSet_bool=False)

# **③DataLoader Class作成**

In [ ]:
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=4,
    num_workers=2)

In [ ]:
# 動作の確認
batch_iterator = iter(data_loader)  # イテレータに変換
dat = next(batch_iterator)  # 1番目の要素を取り出す


In [ ]:
len(data_loader.dataset)

51244

In [ ]:
#print(data_loader.num_workders)

###① candidate_t: 中身のデータ
###② pos_t: 癌 OR Normal
###③ candidateInfo_tup.series_uid: Series_UID
###④ torch.tensor(center_irc): 中心座標のIRC座標

###[32,48,48]の小さな塊の格子の中のデータ
### dat[0]: candidate_t

In [ ]:
print(len(dat))
print(dat[0].shape)

4
torch.Size([4, 1, 32, 48, 48])


###癌または正常の正解ラベル
### dat[1]: post_t

In [ ]:
print(len(dat))
print(dat[1].shape)
print(dat[1])

4
torch.Size([4, 2])
tensor([[1, 0],
        [1, 0],
        [1, 0],
        [1, 0]])


###Series_UID
###dat[2]: user_id

In [ ]:
print(len(dat))
print(dat[2])


4
('1.3.6.1.4.1.14519.5.2.1.6279.6001.216882370221919561230873289517', '1.3.6.1.4.1.14519.5.2.1.6279.6001.724251104254976962355686318345', '1.3.6.1.4.1.14519.5.2.1.6279.6001.138080888843357047811238713686', '1.3.6.1.4.1.14519.5.2.1.6279.6001.525937963993475482158828421281')


###中心のIRC座標
### dat[3]: center_irc

In [ ]:
print(dat[3])

tensor([[229, 272, 370],
        [179, 322, 189],
        [182, 269, 329],
        [342, 284, 238]])


#END